In [1]:
# for Exited, 0 is not-exited, 1 is exited
# for gender, male is 0 and female is 1

#clean outlier data from monthly salary (scale or remove?)

#need to do either undersampling or oversampling, about 8000 users stayed while only 2000 left

#standardscaler or create new features from the features (salary, balance, age, credit score, tenure)

# separate data by region? looks like most data is from france. Much less likely for french customers to leave.
# i.e. the leave data for all three countries is similar but the stay data is much higher in france.

#MY ASSUMPTIONS:
#The data was collected at two points in a year. First point gave balance, salary, etc., then later that year they checked if that person had left or stayed
#Balance may still exist if a person left the bank because it represents the balance they had before leaving
#User is considered an active user if they are still using the bank services in some way. its possible for an active member to leave. its possible for an inactive member to stay.

#is profitable idea: check a 1 for yes and 0 for no, based on iff the user is both an active member AND has a positive balance

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [3]:
starter_df = pd.read_csv(Path("c://python_files/bank_churn/Resources/churn.csv"))
starter_df.sample(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
4119,4120,15588197,Endrizzi,670,France,Male,36,7,0.00,2,0,0,59571.50,0
245,246,15797219,Ifesinachi,635,France,Female,40,10,123497.58,1,1,0,131953.23,1
8119,8120,15586236,Banks,704,France,Male,31,5,132084.66,3,1,1,54474.48,1
2449,2450,15567980,Frater,537,Germany,Female,46,5,100727.50,1,0,1,140857.76,1
3841,3842,15750778,Ponomarev,653,France,Female,60,2,120731.39,4,1,1,138160.11,1
1249,1250,15803681,Sims,803,France,Female,26,4,0.00,2,1,1,181208.47,0
986,987,15655339,Spencer,566,France,Male,36,1,142120.91,1,1,0,79616.37,0
1235,1236,15600700,Pan,523,Germany,Male,63,6,116227.27,1,1,1,119404.63,0
9735,9736,15768495,Chidimma,700,France,Female,32,8,110923.15,2,1,1,161845.81,1
7563,7564,15706394,Howell,609,France,Male,53,7,0.00,2,0,1,52332.85,0


In [4]:
#Convert the values of the Gender column from "M" or "F" to "0" or "1"
starter_df['Gender'] = starter_df['Gender'].str.replace(r'Male', '0', regex=True)
starter_df['Gender'] = starter_df['Gender'].str.replace(r'Female', '1', regex=True)

#Drop the useless columns
starter_df = starter_df.drop(columns=['RowNumber', 'CustomerId', 'Surname'])

In [5]:
starter_df['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [6]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Encode categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(starter_df[['Geography']])

In [7]:
encoded_df_rough = pd.DataFrame(
encoded_data,
columns = enc.get_feature_names(['Geography']))

c:\python_files\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [8]:
starter_df = starter_df.drop(columns=['Geography'])

In [9]:
encoded_df = pd.concat(
    [
        encoded_df_rough,
        starter_df
    ],
    axis=1
)
encoded_df.sample(10)

,Geography_France,Geography_Germany,Geography_Spain,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
4423,1.0,0.0,0.0,785,1,40,3,0.00,2,1,1,96832.82,0
1405,1.0,0.0,0.0,359,1,44,6,128747.69,1,1,0,146955.71,1
5334,0.0,1.0,0.0,786,0,24,2,120135.55,2,1,1,125449.47,0
7681,1.0,0.0,0.0,641,0,33,2,146193.60,2,1,1,55796.83,1
1357,1.0,0.0,0.0,591,0,43,3,0.00,2,0,1,198926.36,0
9653,0.0,1.0,0.0,527,0,58,3,137318.42,1,1,1,126144.96,0
7193,0.0,1.0,0.0,621,1,34,2,91258.52,2,1,0,44857.40,0
92,1.0,0.0,0.0,524,1,36,10,0.00,2,1,0,109614.57,0
4698,1.0,0.0,0.0,732,0,62,2,0.00,2,1,1,25438.87,0
7864,0.0,0.0,1.0,588,0,33,2,0.00,2,1,1,12483.56,0


In [ ]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
main_df.sample(5)